In [ ]:
%pip install pandas numpy scikit-learn rank-bm25 sentence-transformers umap-learn matplotlib seaborn

In [ ]:
# ===============================================
# 5️⃣ Évaluation du moteur de recherche
# ===============================================
import numpy as np

# 🔹 Entrées : 
# embedding_results = {query_id: [top-k IDs de documents récupérés]}
# gts_dict = {query_id: [IDs de documents pertinents]}  # vérité terrain

# Exemple :
# gts_dict = json.load(open("dataset/gts.json"))

k = 10  # Top-k documents à évaluer

# ===============================================
# 🔹 Étape 1 — Precision@k
def precision_at_k(retrieved, relevant, k):
    retrieved_k = retrieved[:k]
    relevant_set = set(relevant)
    hits = sum([1 for doc in retrieved_k if doc in relevant_set])
    return hits / k

# ===============================================
# 🔹 Étape 2 — Recall@k
def recall_at_k(retrieved, relevant, k):
    retrieved_k = retrieved[:k]
    relevant_set = set(relevant)
    hits = sum([1 for doc in retrieved_k if doc in relevant_set])
    return hits / len(relevant_set) if relevant_set else 0

# ===============================================
# 🔹 Étape 3 — Average Precision pour une requête
def average_precision(retrieved, relevant, k):
    relevant_set = set(relevant)
    hits = 0
    sum_precisions = 0
    for i, doc_id in enumerate(retrieved[:k]):
        if doc_id in relevant_set:
            hits += 1
            sum_precisions += hits / (i + 1)
    return sum_precisions / min(len(relevant_set), k) if relevant_set else 0

# ===============================================
# 🔹 Étape 4 — Évaluer toutes les requêtes
precisions, recalls, ap_scores = [], [], []

for query_id, retrieved_docs in embedding_results.items():
    relevant_docs = gts_dict.get(query_id, [])
    
    p = precision_at_k(retrieved_docs, relevant_docs, k)
    r = recall_at_k(retrieved_docs, relevant_docs, k)
    ap = average_precision(retrieved_docs, relevant_docs, k)
    
    precisions.append(p)
    recalls.append(r)
    ap_scores.append(ap)

# ===============================================
# 🔹 Étape 5 — Calculer les métriques moyennes
mean_precision = np.mean(precisions)
mean_recall = np.mean(recalls)
mean_ap = np.mean(ap_scores)  # MAP

print(f"Precision@{k} : {mean_precision:.4f}")
print(f"Recall@{k} : {mean_recall:.4f}")
print(f"MAP@{k} : {mean_ap:.4f}")